In [1]:
using Pkg;
Pkg.activate("..");
Pkg.instantiate();

  Activating project at `~/dev/uni/amo-team-project`


In [2]:
# Pkg.add("Gurobi");
# Pkg.build("Gurobi");
import Gurobi
GRB_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-19


Gurobi.Env(Ptr{Nothing} @0x000056537e27b0c0, false, 0)

In [3]:
# using HiGHS
# using JuMP
# using Ipopt
# using DataFrames
# using JSON
# using Plots
# using StatsPlots
# using StatsBase
# using BenchmarkTools
# using Distributions
# using Distances

In [94]:
include("../src/rwth_parser.jl")
network = read_network("../data/Scenario_2013.xlsx")
loads  = read_loads("../data/Scenario_2013.xlsx")
print()

In [95]:
include("../src/loads.jl")
local_loads = remove_non_data_rows(filter_month(loads, 1))
print()

In [96]:
include("../src/scenarios/generation.jl")
attack_scenarios = generate_attack_scenarios(network, 10, 1000)
weather_scenarios = generate_weather_scenarios(1000)
print()

In [97]:
include("../src/scenarios/reduction.jl")
reduced_attack_scenarios = reduce_binary_scenarios(attack_scenarios, 4)
reduced_weather_scenarios = reduce_continous_scenarios(weather_scenarios, 4)
reduced_load_scenarios = reduce_continous_scenarios(local_loads, 4)
print()

In [98]:
include("../src/scenarios/assemble.jl")
scenarios = cartesian_scenarios(
	reduced_load_scenarios,
	reduced_attack_scenarios,
	reduced_weather_scenarios,
)
print()

In [99]:
scenarios = translate_weather_to_capacity(scenarios, network.generators)
print()

In [100]:
scenario_dict = convert_df_to_scenarios(scenarios)
print("Number of scenarios: $(length(scenario_dict))")

Number of scenarios: 64

In [101]:
include("../src/model.jl")
dispatch_model = DispatchModel(network, scenario_dict, 5.0)
init_model!(dispatch_model)
print()

In [102]:
dispatch_model.m

A JuMP Model
Minimization problem with:
Variables: 41016
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 20992 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 4608 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 69889 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 7680 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 7800 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: reinforcement_budget

In [103]:
t = @time solve!(dispatch_model)
print()

 36.405857 seconds (822.61 k allocations: 67.515 MiB)


In [105]:
3*12*36 / 60

21.6

In [75]:
import JuMP
print("Objective value: $(JuMP.objective_value(dispatch_model.m))")

Objective value: 1166.4784488866733

In [78]:
include("../src/model.jl")
dispatch_model = DispatchModel(network, scenario_dict, 5.0)
t = @time solve_with_benders!(dispatch_model)
print()

In [77]:
JuMP.objective_value(dispatch_model.m)

2931.9324380360194

In [46]:
vars = dispatch_model.variables[:mp]
print()

In [52]:
vars = dispatch_model.variables[Symbol()]
print()

In [47]:
include("../src/results.jl")

store_results_in_dir (generic function with 1 method)

In [56]:
results_sum = get_results_summary(dispatch_model)
results_sum.objective_value

1121.7745501867353

In [55]:
results_r = convert_jump_container_to_df(vars[:r])
results_r[results_r.Value .!= 0, :]

Row,dim1,Value
,Symbol,Float64
1,B12,1.0
2,B24,1.0
3,B26,1.0
4,B38,1.0
5,B90,1.0


In [88]:
p_results = reformat_2d_df(convert_jump_container_to_df(vars[:P]))

Row,dim1,S1,S2
,Symbol,Float64?,Float64?
1,G1,1000.0,0.0
2,G10,177.0,371.508
3,G11,282.0,600.0
4,G12,551.0,551.0
5,G13,1600.0,683.0
6,G14,378.0,378.0
7,G15,800.0,395.0
8,G16,870.388,0.0
9,G17,488.0,0.0


In [89]:
l_shed_results = reformat_2d_df(convert_jump_container_to_df(vars[:L_shed]))
any_rows_greater_zero(l_shed_results)

Row,dim1,S1,S2
,Symbol,Float64?,Float64?
1,B105,177.37,0.0
2,B116,157.19,0.0
3,B24,399.558,0.0
4,B41,0.0,152.184
5,B42,0.0,148.357
6,B48,135.285,0.0
7,B51,0.0,147.859
8,B56,0.0,116.493
9,B57,0.0,120.692


In [90]:
z_results = reformat_2d_df(convert_jump_container_to_df(vars[:z]))
any_rows_equal_zero(z_results)

Row,dim1,S1,S2
,Symbol,Float64?,Float64?
1,B10,0.0,0.0
2,B105,-0.0,1.0
3,B116,0.0,1.0
4,B119,0.0,0.0
5,B120,0.0,0.0
6,B24,-0.0,1.0
7,B41,1.0,-0.0
8,B42,1.0,0.0
9,B48,-0.0,1.0


In [91]:
attacked_busses_df = get_attacked_busses_df(scenarios)
any_rows_greater_zero(attacked_busses_df)

Row,scenario,S1,S2
,String,Int64,Int64
1,B90_attack,1,0
2,B12_attack,1,0
3,B78_attack,1,0
4,B105_attack,1,0
5,B56_attack,0,1
6,B38_attack,0,1
7,B68_attack,1,0
8,B26_attack,0,1
9,B116_attack,1,0
